In [ ]:
pip install hmmlearn

     |████████████████████████████████| 368kB 5.6MB/s 


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
from datetime import datetime
import calendar
import math
from hmmlearn import hmm

In [ ]:
df_1 = pd.read_csv ("/content/drive/My Drive/reddit_bigger_dataset/dataset/depression.csv")
df_2 = pd.read_csv ("/content/drive/My Drive/reddit_bigger_dataset/dataset/suicidewatch.csv")

In [ ]:
df_List = [df_1 , df_2]  
for data_frame in df_List : 
  data_frame['index_indi'] = pd.Series ([i for i in range (data_frame.shape[0])])

In [ ]:
df = pd.concat(df_List, ignore_index=True)

In [ ]:
df.shape

(130539, 5)

In [ ]:
for i in range (df.date.shape[0]) : 
  df.date[i] = datetime.strptime(df.date[i], '%Y-%m-%d %H:%M:%S')
df.sort_values(by=['date'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.head()

,subreddit,author,date,post,index_indi
0,depression,bumblebeehoneycomb,2020-01-01 05:30:56,I know as well as everyone new years makes you...,0
1,depression,anonloz11,2020-01-01 05:31:25,"I don’t know about anyone else, but I’m a litt...",1
84591,SuicideWatch,AJSKFAQ,2020-01-01 05:31:37,Happy new year nonetheless,0
84592,SuicideWatch,sikpenfold,2020-01-01 05:31:50,NaN,1
2,depression,EuphoricSorrow91,2020-01-01 05:32:44,"Good f\*\*king riddance, 2010s decade.\n\n&amp...",2


In [ ]:
unique_name = df.author.unique().tolist()
unique_name = np.unique(np.array(unique_name))

In [ ]:
post_per_person = []
for k, name in enumerate (unique_name) :
  post_counts = df.loc[df['author'] == name].shape[0]
  post_per_person.append ( [name, int (post_counts)] )

In [ ]:
temp = []
for i in range (len(post_per_person)) : 
  temp.append (post_per_person[i][1])
sum (temp) / len(post_per_person)

1.7746886725759965

In [ ]:
sub_data_names = [] 
for i, freq in enumerate (temp) : 
  if freq >= 5 and freq <= 50 : 
    sub_data_names.append (post_per_person[i][0])

In [ ]:
sub_data_set = [] 
for name in sub_data_names : 
  sub_data_set.append (df.loc[df.author == name])

In [ ]:
train_cases = []
for i in range(len ( sub_data_set) ) : 
    cat = sub_data_set[i]['subreddit'].to_numpy()
    if 'SuicideWatch' in cat and 'depression' in  cat: 
      temp_1 = sub_data_set[i][sub_data_set[i].subreddit == 'SuicideWatch']
      temp_2 = sub_data_set[i][sub_data_set[i].subreddit == 'depression']
      sub_data_set[i] = pd.concat([temp_1, temp_2])
      sub_data_set[i].sort_values(by=['date'], inplace=True)
      train_cases.append (sub_data_set[i])

In [ ]:
Use_this = [] 
Labels = [] 
for i in range (len(train_cases)) : 
  Use_this.append (train_cases[i].post.to_numpy())
  Labels.append (train_cases[i].subreddit.to_numpy())

In [ ]:
Use_this[0] , Labels[0]

(array(['I watched the documentary called “The Bridge” (on YouTube) about people jumping off the golden bridge and it gave me a sort of happiness and relief...that I could go there and simply jump off and I would die just like that without really processing the pain. I feel so motivated to go there and do it, it’s giving me adrenaline.\n\nI also find it quite “romantic” (I think that’s the word) knowing the fact that I’m leaving the world behind me and likely never being able to experience what the world has to offer ever again. It almost feels like I get to go in a different dimension while everyone is stuck on earth.\n\nNgl I feel quite pumped and happy right now.',
        'Yea, I have slightly more energy, appetite and sleep better BUT I’M STILL FUCKING EMOTIONALLY NUMB. I don’t even care about having a appetite or being slightly more energetic. \n\nI. Just. Want. Not. To. Be. Dead. Inside.  Is that too much to ask? Every fucking day I do my regular shit like wake up and go to scho

In [ ]:
Sum = 0 
for i in range (len(Use_this)) : 
  Sum += len(Use_this[i])
Sum

14259

In [ ]:
LIWC_depression = pd.read_csv ( "/content/drive/MyDrive/reddit_bigger_dataset/liwcfinal/LIWC2015 Results (depression (84591 files)).csv")
LIWC_suicide = pd.read_csv ( "/content/drive/MyDrive/reddit_bigger_dataset/liwcfinal/LIWC2015 Results (suicidewatch (45948 files)).csv")

LIWC_df = pd.concat([ LIWC_depression , LIWC_suicide])

In [ ]:
LIWC_suicide.head()

,Filename,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,suicidewatch0.txt,1.0,4.0,38.60,50.00,99.00,99.00,4.0,25.00,100.0,25.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,25.00,0.00,0.0,0.00,50.00,0.00,0.00,0.00,0.00,25.00,25.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,0.0,0.00,50.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
1,suicidewatch1.txt,1.0,1.0,93.26,50.00,1.00,25.77,1.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
2,suicidewatch10.txt,1.0,12.0,62.04,99.00,23.51,25.77,12.0,25.00,100.0,50.00,25.00,16.67,8.33,8.33,0.00,0.0,0.0,8.33,8.33,8.33,0.0,8.33,0.00,0.0,25.00,8.33,8.33,0.00,0.00,8.33,16.67,8.33,8.33,0.0,8.33,0.00,33.33,0.0,0.0,...,0.0,0.00,0.0,25.00,16.67,8.33,8.33,8.33,0.00,0.00,16.67,8.33,16.67,0.0,16.67,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,33.33,8.33,16.67,0.0,0.0,0.00,0.0,0.0,0.0,8.33,0.0,0.0
3,suicidewatch100.txt,1.0,41.0,27.63,23.22,71.40,99.00,8.2,9.76,87.8,53.66,21.95,14.63,9.76,0.00,4.88,0.0,0.0,7.32,7.32,9.76,12.2,9.76,2.44,0.0,21.95,9.76,4.88,2.44,7.32,0.00,17.07,12.20,4.88,0.0,2.44,2.44,4.88,0.0,0.0,...,0.0,2.44,0.0,4.88,0.00,2.44,4.88,0.00,2.44,4.88,12.20,4.88,14.63,0.0,12.20,2.44,2.44,0.0,0.0,0.0,0.0,0.0,4.88,2.44,0.0,0.0,2.44,0.0,29.27,9.76,7.32,0.0,0.0,2.44,0.0,0.0,0.0,9.76,0.0,0.0
4,suicidewatch1000.txt,1.0,5.0,8.69,99.00,43.37,99.00,5.0,0.00,100.0,40.00,40.00,20.00,20.00,0.00,0.00,0.0,0.0,20.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,20.00,20.00,0.00,0.0,0.00,0.00,60.00,0.0,0.0,...,0.0,0.00,0.0,40.00,40.00,0.00,40.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,20.00,0.00,20.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0


In [ ]:
LIWC_df.head()

,Filename,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,depression0.txt,1.0,70.0,2.47,90.06,78.40,93.21,17.50,7.14,98.57,60.00,20.00,11.43,4.29,1.43,5.71,0.00,0.00,8.57,2.86,7.14,11.43,12.86,14.29,0.00,22.86,14.29,5.71,2.86,1.43,1.43,10.00,7.14,2.86,0.00,0.00,2.86,12.86,0.0,0.00,...,0.00,0.00,0.0,5.71,1.43,1.43,0.00,2.86,1.43,1.43,18.57,1.43,17.14,0.00,5.71,12.86,0.00,0.00,1.43,1.43,0.0,0.00,2.86,0.00,0.00,0.00,2.86,0.0,11.43,7.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.29,0.00,0.00
1,depression1.txt,1.0,84.0,48.63,2.86,99.00,1.00,12.00,5.95,94.05,55.95,17.86,9.52,9.52,0.00,0.00,0.00,0.00,8.33,8.33,13.10,9.52,5.95,5.95,3.57,20.24,7.14,2.38,1.19,2.38,5.95,7.14,1.19,5.95,1.19,0.00,3.57,1.19,0.0,0.00,...,0.00,0.00,0.0,3.57,0.00,1.19,2.38,0.00,0.00,4.76,13.10,1.19,25.00,1.19,8.33,16.67,0.00,0.00,0.00,0.00,0.0,0.00,1.19,0.00,0.00,1.19,0.00,0.0,22.62,10.71,5.95,0.00,0.00,0.00,0.00,0.00,0.00,5.95,0.00,0.00
2,depression10.txt,1.0,243.0,38.32,80.62,49.92,19.62,15.19,14.40,88.89,59.67,17.28,5.76,0.00,0.00,5.76,0.00,0.00,11.11,7.82,12.35,11.93,5.76,8.64,1.65,16.87,5.35,2.47,1.65,0.00,0.82,4.53,2.06,2.47,0.82,0.00,1.23,9.47,0.0,0.00,...,0.00,0.00,3.7,2.88,0.00,0.41,1.23,1.65,0.00,1.65,13.99,1.23,16.05,2.47,6.17,7.82,0.00,4.94,0.41,0.00,0.0,0.41,0.41,0.00,0.41,0.00,0.00,0.0,41.98,8.23,12.35,0.41,0.00,0.41,0.82,1.23,0.82,2.06,0.82,14.81
3,depression100.txt,1.0,35.0,36.68,7.67,78.40,1.00,11.67,25.71,94.29,42.86,20.00,14.29,14.29,0.00,0.00,0.00,0.00,5.71,0.00,8.57,0.00,5.71,5.71,2.86,17.14,11.43,2.86,2.86,2.86,0.00,22.86,5.71,17.14,0.00,17.14,0.00,11.43,0.0,0.00,...,0.00,8.57,0.0,2.86,0.00,0.00,0.00,2.86,0.00,2.86,17.14,0.00,8.57,2.86,2.86,2.86,0.00,0.00,0.00,0.00,0.0,5.71,11.43,8.57,0.00,2.86,0.00,0.0,5.71,5.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,depression1000.txt,1.0,626.0,18.72,33.88,81.40,3.25,11.18,13.42,95.21,62.78,23.80,16.61,12.46,0.00,0.48,3.35,0.32,7.03,5.11,13.26,12.46,6.39,6.55,2.56,21.57,5.43,2.24,2.72,0.64,1.44,5.91,1.76,4.15,0.16,1.12,2.40,14.70,2.4,1.44,...,1.76,0.32,0.0,7.83,2.88,1.92,1.76,1.60,0.32,9.11,10.70,0.80,13.42,1.60,5.11,6.87,1.28,0.32,0.16,0.32,0.0,1.28,0.32,0.16,0.00,0.16,0.00,0.0,15.02,8.79,0.16,0.00,0.32,0.16,0.00,0.16,0.32,4.79,0.00,0.32


In [ ]:
def check_for_nan (data):
  for j, k in enumerate ( data ) : 
    if math.isinf (k) or np.isnan (k) : 
      data[j] = 0 
  return data 

In [ ]:
col = LIWC_df.columns[2:]

LIWC_sequence = []

for case in train_cases[:-100] : 
  temp = []
  for index, cat in zip ( case.index_indi.to_numpy(), case.subreddit.to_numpy() ): 
    if cat == 'SuicideWatch' : 
      file_name = "suicidewatch" + str (index) + ".txt" 
    elif cat == 'depression' : 
      file_name = "depression" + str (index) +".txt" 
    liwc_vec = LIWC_df[LIWC_df.Filename == file_name][col].to_numpy().flatten()
    liwc_vec = check_for_nan (liwc_vec)

    if liwc_vec.shape[0] > 0:  
      temp.append (liwc_vec)
  # if len (temp) >= 5: 
  LIWC_sequence.append (temp)  

In [ ]:
lengths = []
X = []
for i in range (len(LIWC_sequence)) : 
  lengths.append (len(LIWC_sequence[i]))  
  X += LIWC_sequence[i]

In [ ]:
model = hmm.GaussianHMM(2, "full", algorithm='viterbi')
model.fit (X, lengths)

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=2, n_iter=10, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=False)

In [ ]:
bool_labels = []
for L in Labels :  
  temp = []
  for i in L :
    if i == 'depression' : 
      temp.append (0) 
    else : 
      temp.append (1) 
  bool_labels.append (temp) 

In [ ]:
def evaluation (prediction, test):
  cp_per_sequence = []
  ct_per_sequence = [] 
  Sum = 0 
  for Y_i, Y_h in zip(prediction, test) : 
    cp = 0 
    ct = 0 
    for i , y_i in enumerate (Y_i) :
      if Y_h[i] == y_i : 
        cp += 1 
      if Y_h[i] == y_i and Y_h[i-1] == Y_i[i-1] and Y_h[i] != Y_h[i-1]  : 
        ct += 1 
    cp_per_sequence.append (cp) 
    ct_per_sequence.append (ct)

  return cp_per_sequence , ct_per_sequence 

In [ ]:

prev = 0   
new = lengths[0]
indices = []
for i in range (1, len(lengths)) : 
  indices.append ((prev , new))
  prev = new
  new += lengths[i]
indices.append ((prev, new))

In [ ]:
prediction = [] 
for i in range (len(indices)-100, len(indices)) : 
  prediction.append (model.predict(X[indices[i][0]:indices[i][1]]))
  

In [ ]:
r_1, r_2 = evaluation (prediction , bool_labels[len(indices)-100:len(indices)]) 

In [ ]:
tc_real = []
for L in bool_labels : 
  c = 0
  for i in range (1, len(L)) :
    if L[i-1] != L[i] : 
      c += 1 
  tc_real.append (c)

In [ ]:
print ("accuracy for individual labels : " , sum (r_1)/sum (lengths[len(indices)-100:len(indices)]) )

accuracy for individual labels :  0.5353260869565217


In [ ]:
print ("accuracy for transitions : " , sum (r_2) / sum (tc_real[len(indices)-100:len(indices)]) )

accuracy for transitions :  0.1970074812967581
